In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Modelling")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '8g')
    .config('spark.executor.memory', '8g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/19 07:38:44 WARN Utils: Your hostname, flgg resolves to a loopback address: 127.0.1.1; using 172.28.210.36 instead (on interface eth0)
23/08/19 07:38:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 07:38:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/19 07:38:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/19 07:38:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
test_pd = spark.read.parquet('../data/raw/2023/yellow_tripdata_2023-04.parquet')

In [3]:
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import col, hour
from pyspark.sql.functions import when
from pyspark.sql import functions as F
from pyspark.sql.functions import date_format

In [4]:
test_pd = test_pd.withColumn("time_set", when((hour(col("tpep_pickup_datetime")) >= 5) & (hour(col("tpep_pickup_datetime")) < 12), "Morning")\
.when((hour(col("tpep_pickup_datetime")) >= 12) & (hour(col("tpep_pickup_datetime")) < 18), "Afternoon")\
.when((hour(col("tpep_pickup_datetime")) >= 18) & (hour(col("tpep_pickup_datetime")) < 23), "Evening")\
.otherwise("Late Night"))\
.withColumn('time_duration_minutes', (unix_timestamp('tpep_dropoff_datetime') - unix_timestamp('tpep_pickup_datetime')) / 60)\
.withColumn('average_speed', F.col('trip_distance') / (F.col('time_duration_minutes') / 60))\
.withColumn('Month', date_format('tpep_pickup_datetime','MM').cast('int'))

test_pd = test_pd.where(F.col('time_duration_minutes') >= 1).where(F.col('tip_amount') > 0)\
.where(F.col('trip_distance') > 0).where(F.col('payment_type') == 1)\
.where(F.col('fare_amount') >= 3)

In [7]:
test_pd.write.parquet("../data/curated/test_first_clean")

In [8]:
test_df = spark.read.parquet('../data/curated/test_first_clean')

In [9]:

def compute_and_save(df):
    # 使用Spark的groupBy和agg功能来同时计算多个聚合
    grouped_data = (df.groupBy("PULocationID")
                     .agg(F.sum("tip_amount").alias("total_tip"),
                          F.sum("fare_amount").alias("total_fare"),
                          F.sum("time_duration_minutes").alias("total_time"),
                          F.count("*").alias("count")))

    # 计算平均值和比率
    result_df = (grouped_data.withColumn("average_tip", F.col("total_tip") / F.col("count"))
                             .withColumn("average_fare", F.col("total_fare") / F.col("count"))
                             .withColumn("average_time", F.col("total_time") / F.col("count"))
                             .withColumn("tip_rate", F.col("average_tip") / F.col("average_fare")))

    # 选择我们需要的列
    result_df = result_df.select("PULocationID", "average_tip", "average_fare", "tip_rate", "average_time")

    # 保存到csv文件
    result_df.write.csv("../data/curated/test_final.csv", header=True)

    return result_df


compute_and_save(test_df)


PULocationID,average_tip,average_fare,tip_rate,average_time
148,4.066362784421073,17.51744573977587,0.2321321752513161,14.840832302482863
243,6.224621212121215,28.638939393939403,0.21734817503187548,19.29381313131314
31,15.765,63.2,0.24944620253164557,34.80833333333334
137,3.4575379727217355,14.244034795412386,0.24273585556216937,13.216728017152333
85,5.48,25.633333333333336,0.21378413524057216,16.35
65,5.741073345259386,25.098014311270127,0.22874611808159612,21.457245080500854
255,4.707667560321718,21.09932975871312,0.22311929403243969,17.951988382484377
53,9.447777777777778,41.03333333333333,0.2302464121310588,30.187037037037037
133,6.739999999999999,27.441666666666674,0.24561190403887023,19.022222222222222
78,8.615,31.0,0.2779032258064516,23.4


In [10]:
test = spark.read.csv('../data/curated/test_final.csv')

In [12]:
test = test.toPandas()
test.columns = test.iloc[0]


# 删除第一行和第二行
test = test.drop([0, 1]).reset_index(drop=True)
test = test.rename(columns={'PULocationID': 'LocationID'})


In [13]:
test

,LocationID,average_tip,average_fare,tip_rate,average_time
0,243,6.224621212121215,28.638939393939403,0.21734817503187548,19.29381313131314
1,31,15.765,63.2,0.24944620253164557,34.80833333333334
2,137,3.4575379727217355,14.244034795412386,0.24273585556216937,13.216728017152333
3,85,5.48,25.633333333333336,0.21378413524057216,16.35
4,65,5.741073345259386,25.098014311270127,0.22874611808159612,21.457245080500854
...,...,...,...,...,...
230,180,12.449,56.89000000000001,0.218825804183512,24.403333333333332
231,134,14.152857142857128,65.05057142857144,0.21756699183492376,17.55968253968253
232,36,5.266981132075471,26.16981132075472,0.20126171593366973,20.22389937106918
233,208,7.553333333333334,52.23333333333333,0.14460753031269946,23.416666666666668


In [14]:
test['tip_rate'] = test['tip_rate'].astype(float)
test['average_time'] = test['average_time'].astype(float)

In [15]:
test.to_csv('../data/curated/test_pd.csv', index=False)

In [17]:
import pandas as pd

In [18]:
test_pd = pd.read_csv('../data/curated/test_pd.csv')
test_df_crime = pd.read_csv('../data/curated/test_df_crime.csv')

In [20]:
import pandas as pd
import geopandas as gpd

sf = gpd.read_file("../data/landing/taxi_zones.shp")
zones = pd.read_csv("../data/landing/taxi+_zone_lookup.csv")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")


gdf = gpd.GeoDataFrame(
    pd.merge(zones, sf, on='LocationID', how='inner')
)

In [21]:
merged_df_test = pd.merge(test_pd, test_df_crime, on='LocationID', how='inner')
merged_df_test = pd.merge(merged_df_test, gdf, on='LocationID', how='inner')
merged_df_test

,LocationID,average_tip,average_fare,tip_rate,average_time,Unnamed: 0,crime_count,Borough,Zone,service_zone,OBJECTID,Shape_Leng,Shape_Area,zone,borough,geometry
0,243,6.224621,28.638939,0.217348,19.293813,235,274,Manhattan,Washington Heights North,Boro Zone,243,0.094331,0.000438,Washington Heights North,Manhattan,"POLYGON ((-73.93157 40.86958, -73.93141 40.868..."
1,31,15.765000,63.200000,0.249446,34.808333,28,51,Bronx,Bronx Park,Boro Zone,31,0.096425,0.000334,Bronx Park,Bronx,"POLYGON ((-73.87094 40.85727, -73.87094 40.857..."
2,137,3.457538,14.244035,0.242736,13.216728,130,108,Manhattan,Kips Bay,Yellow Zone,137,0.046108,0.000116,Kips Bay,Manhattan,"POLYGON ((-73.97220 40.74040, -73.97222 40.740..."
3,85,5.480000,25.633333,0.213784,16.350000,81,173,Brooklyn,Erasmus,Boro Zone,85,0.057475,0.000144,Erasmus,Brooklyn,"POLYGON ((-73.94716 40.65559, -73.94703 40.654..."
4,65,5.741073,25.098014,0.228746,21.457245,61,246,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,65,0.044607,0.000082,Downtown Brooklyn/MetroTech,Brooklyn,"POLYGON ((-73.98712 40.70063, -73.98706 40.700..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,180,12.449000,56.890000,0.218826,24.403333,173,105,Queens,Ozone Park,Boro Zone,180,0.080528,0.000248,Ozone Park,Queens,"POLYGON ((-73.84120 40.68335, -73.84013 40.681..."
226,134,14.152857,65.050571,0.217567,17.559683,127,61,Queens,Kew Gardens,Boro Zone,134,0.069588,0.000202,Kew Gardens,Queens,"POLYGON ((-73.82289 40.71193, -73.82253 40.711..."
227,36,5.266981,26.169811,0.201262,20.223899,33,312,Brooklyn,Bushwick North,Boro Zone,36,0.087051,0.000248,Bushwick North,Brooklyn,"POLYGON ((-73.91255 40.70387, -73.91181 40.703..."
228,208,7.553333,52.233333,0.144608,23.416667,200,204,Bronx,Schuylerville/Edgewater Park,Boro Zone,208,0.214721,0.002020,Schuylerville/Edgewater Park,Bronx,"MULTIPOLYGON (((-73.82599 40.84068, -73.82597 ..."


In [22]:
merged_df_test.to_csv('../data/curated/merged_df_test.csv')